 Alocação de salas para disciplinas (timetabling simplificado)

In [ ]:
# Implementação completa do resolvedor (greedy + backtracking)
from collections import defaultdict

class Sala:
    def __init__(self, id, capacidade):
        # identificador da sala
        self.id = id
        # capacidade máxima de alunos
        self.capacidade = capacidade
        # conjunto de horários ocupados nesta sala (strings)
        self.horarios_ocupados = set()
    def __repr__(self):
        return f"Sala({self.id}, cap={self.capacidade})"

class Disciplina:
    def __init__(self, id, num_alunos, professor):
        # identificador único (string)
        self.id = id
        # número de alunos matriculados
        self.num_alunos = num_alunos
        # nome do professor (usado para conflito de horário)
        self.professor = professor
    def __repr__(self):
        return f"Disciplina({self.id}, alunos={self.num_alunos}, prof={self.professor})"

class Scheduler:
    def __init__(self, salas, disciplinas, timeslots):
        self.salas = salas
        self.disciplinas = disciplinas
        self.timeslots = timeslots

    def can_allocate(self, disciplina, sala, horario, alocacoes, prof_busy):
        # Capacidade suficiente?
        if sala.capacidade < disciplina.num_alunos:
            return False
        # Horário livre na sala?
        if horario in sala.horarios_ocupados:
            return False
        # Professor livre neste horário?
        if disciplina.professor in prof_busy.get(horario, set()):
            return False
        return True

    def assign(self, disciplina, sala, horario, alocacoes, prof_busy):
        alocacoes[disciplina.id] = (sala.id, horario)
        sala.horarios_ocupados.add(horario)
        prof_busy.setdefault(horario, set()).add(disciplina.professor)

    def unassign(self, disciplina, sala, horario, alocacoes, prof_busy):
        # Remove a alocação e estados auxiliares
        if disciplina.id in alocacoes:
            del alocacoes[disciplina.id]
        if horario in sala.horarios_ocupados:
            sala.horarios_ocupados.remove(horario)
        if horario in prof_busy and disciplina.professor in prof_busy[horario]:
            prof_busy[horario].remove(disciplina.professor)
            if not prof_busy[horario]:
                del prof_busy[horario]

    def _select_unassigned(self, alocacoes):
        # Heurística simples: escolher disciplina com MAIOR número de alunos (mais difícil)
        unassigned = [d for d in self.disciplinas if d.id not in alocacoes]
        unassigned.sort(key=lambda d: d.num_alunos, reverse=True)
        return unassigned[0] if unassigned else None

    def backtrack(self, alocacoes, prof_busy):
        # Se todas alocadas, devolve solução (cópia)
        if len(alocacoes) == len(self.disciplinas):
            return dict(alocacoes)

        disciplina = self._select_unassigned(alocacoes)
        # tenta salas e horários (ordena salas por capacidade crescente para melhor ajuste)
        for sala in sorted(self.salas, key=lambda s: s.capacidade):
            for horario in self.timeslots:
                if self.can_allocate(disciplina, sala, horario, alocacoes, prof_busy):
                    self.assign(disciplina, sala, horario, alocacoes, prof_busy)
                    sol = self.backtrack(alocacoes, prof_busy)
                    if sol is not None:
                        return sol
                    # desfazer e continuar buscando
                    self.unassign(disciplina, sala, horario, alocacoes, prof_busy)
        # sem solução neste ramo
        return None

    def solve_backtracking(self):
        # reseta estados das salas antes de buscar
        for s in self.salas:
            s.horarios_ocupados = set()
        return self.backtrack({}, {})

    def solve_greedy(self):
        # heurística gulosa: ordena disciplinas por tamanho e tenta colocar na primeira opção válida
        for s in self.salas:
            s.horarios_ocupados = set()
        alocacoes = {}
        prof_busy = {}
        for disciplina in sorted(self.disciplinas, key=lambda d: d.num_alunos, reverse=True):
            placed = False
            for horario in self.timeslots:
                for sala in sorted(self.salas, key=lambda s: s.capacidade):
                    if self.can_allocate(disciplina, sala, horario, alocacoes, prof_busy):
                        self.assign(disciplina, sala, horario, alocacoes, prof_busy)
                        placed = True
                        break
                if placed:
                    break
            if not placed:
                return None  # falha gulosa
        return alocacoes

# Exemplo de uso: definir horários, salas e disciplinas e rodar os dois métodos
if __name__ == '__main__':
    timeslots = ['08:00-10:00', '10:00-12:00', '14:00-16:00']
    salas = [Sala('Sala1', 30), Sala('Sala2', 50), Sala('Sala3', 20)]
    disciplinas = [
        Disciplina('Math', 25, 'ProfA'),
        Disciplina('Physics', 40, 'ProfB'),
        Disciplina('Chem', 18, 'ProfC'),
        Disciplina('Bio', 20, 'ProfA'),
    ]
    scheduler = Scheduler(salas, disciplinas, timeslots)

    print('Tentativa gulosa:')
    sol_greedy = scheduler.solve_greedy()
    print(sol_greedy)

    print('





    # substitua a heurística em _select_unassigned por contagem de conflitos.    # Observação: se desejar priorizar conflitos por professor ou por grau,    print(sol_bt)    sol_bt = scheduler.solve_backtracking()Tentativa backtracking (exata):')